# Please upvote for further encouragement.

**SentiWordNet is a library created by NLP community engineers for sentiment analysis.**

It contains POS Tags, Positivity score, Negativity score and SynsetTerms ie. in common language synonyms of a perticular words.
As scores related to words are already define in SentiWordNet based on some rules **and we are not considering context or sequence of word in which a word is used**, This approach is called as Rule Based Sentiment Analysis Approach.

# Importing Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import os
import string
import re
import warnings 
warnings.filterwarnings('ignore')

#plotting libraries!
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
%matplotlib inline
#PLOTLY
import plotly
import plotly.offline as offline
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import cufflinks as cf
from collections import defaultdict
from plotly import tools
from nltk.util import ngrams
from plotly.graph_objs import Scatter, Figure, Layout
cf.set_config_file(offline=True)
from nltk.corpus import stopwords
eng_stopwords = set(stopwords.words("english"))
from wordcloud import WordCloud, STOPWORDS
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import squarify
from PIL import Image
from os import path
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from textblob import TextBlob
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords

nltk.download('wordnet')
nltk.download('sentiwordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer()

In [ ]:
!pip install tweet-preprocessor
import preprocessor as p
p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.SMILEY, p.OPT.EMOJI)

In [ ]:
#for suppressing DeprecationWarnings:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
df = pd.read_csv('../input/reddit-vaccine-myths/reddit_vm.csv')

In [ ]:
df.head(5)

In [ ]:
print(df.shape)

**The TextBlob Sentiment Analysis of TextBlob returns two properties**

* Polarity
* Subjectivity
* Polarity
It simply means emotions expressed in a sentence.

* Emotions are closely related to sentiments. The strength of a sentiment or opinion is typically linked to the intensity of certain emotions, e.g., joy and anger.
* The range of the Polarity lies in [-1.0,1.0] where 1 is a positive statment and -1 is negative statement. Values with 0 is Neutral statment

**Subjectivity**
* Subjective sentence expresses some personal individual feelings, views, or beliefs.
* The range of Subjectivity is [0.0,1.0] where 0 is very objective and 1 is very Subjective.

In [ ]:
print(df['title'][2])
Tweet = TextBlob(df['title'][4])
Tweet.sentiment

Get score of each tweet - 
SentiWordNet is a library created by NLP community engineers for sentiment analysis.
It contains POS Tags, Positivity score, Negativity score and SynsetTerms ie. in common language synonyms of a perticular words.
As scores related to words are already define in SentiWordNet based on some rules [and we are not considering context or sequence of word in which a word is used], This approach is called as Rule Based Sentiment Analysis Approach.

# Text PreProcessing
* Removing Punctuations
* Change Case
* Tokenization
* Removing Stop Words
* Lemmatization

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
cleantweets = []
postags = []
import string
table = str.maketrans('','', string.punctuation)
for title in df['title']:
    try:
        title = p.clean(title)
        #tokenize + lower case
        words2 = word_tokenize(title.lower())
        #remove puncts
        words3 = [w.translate(table) for w in words2]
        #remove stopwords
        words4 = [word for word in words3 if word not in stop_words]
        #applying lemmatization
        words5 = [lemmatizer.lemmatize(word) for word in words4]
                
        #combining all words
        cleantweets.append((" ".join(words5)).strip())
    except:
        cleantweets.append(title)
        continue

print(len(cleantweets))
df['clean_Tweets'] = cleantweets

# POS Tagging

In [ ]:
pos=neg=obj=count=0

postagging = []

for title in df['clean_Tweets']:
    list = word_tokenize(title)
    postagging.append(nltk.pos_tag(list))

df['pos_tags'] = postagging

# Functions for Sentiment Scoring

In [ ]:
# Convert between the PennTreebank tags to simple Wordnet tags
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [ ]:
def get_sentiment(word,tag):
    wn_tag = penn_to_wn(tag)
    
    if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
        return []

    #Lemmatization
    lemma = lemmatizer.lemmatize(word, pos=wn_tag)
    if not lemma:
        return []

    #Synset is a special kind of a simple interface that is present in NLTK to look up words in WordNet. 
    #Synset instances are the groupings of synonymous words that express the same concept. 
    #Some of the words have only one Synset and some have several.
    synsets = wn.synsets(word, pos=wn_tag)
    if not synsets:
        return []

    # Take the first sense, the most common
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name())

    return [synset.name(), swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score()]

In [ ]:
pos=neg=obj=count=0
senti_score = []

for pos_val in df['pos_tags']:
    senti_val = [get_sentiment(x,y) for (x,y) in pos_val]
    for score in senti_val:
        try:
            pos = pos + score[1]  #positive score is stored at 2nd position
            neg = neg + score[2]  #negative score is stored at 3rd position
        except:
            continue
    senti_score.append(pos - neg)
    pos=neg=0    
    
df['senti_score'] = senti_score

In [ ]:
df.head(5)

SentiWordNet is a lexical resource for opinion mining. SentiWordNet assigns to each synset of WordNet three sentiment scores: positivity, negativity, objectivity.

In [ ]:
df.to_csv("Sentiment_predited_result.csv")

In [ ]:
df['compound_label'] = df['senti_score'].apply(lambda number: 'positive' if number>=0 else 'negative')

In [ ]:
df.head(5)

In [ ]:
df['compound_label'].value_counts().plot(kind='pie')

In [ ]:
text = df['clean_Tweets']

In [ ]:
wc = WordCloud(width = 900, height = 500, background_color = 'white', random_state = 10).generate(text[1])

plt.title('Word Cloud for Bag Of Words')
plt.imshow(wc)
plt.axis('off')
plt.show()

**Sentence tokenization is the process of splitting text into individual sentences**

In [ ]:
sents1 = sent_tokenize(text	[5])
print(f'Sentence Tokenization using NLTK: \n {text	[5]} => {sents1}')

****It is used on words so that individual words are separated as items in a list.

In [ ]:
words1 = word_tokenize(sents1[0])
print(f'Word Tokenization using NLTK: \n {sents1[0]} => {words1}')

In [ ]:
tokens1 = [word for word in words1 if not word in stopwords.words('english')] 
print(f'Stopword Removal using NLTK: \n{words1} => {tokens1}')

****N-gram is a language model widely used in NLP and is applied to statistical problems involving text and audio. It is a probabilistic model that predicts the next series of words. For example, in the sentence, “The movie was boring.” 

In [ ]:
n_grams1 = ngrams(tokens1, 2)
n_grams1 = [ ' '.join(grams) for grams in n_grams1]

print(f'N-Gram using NLTK (n = 2): \n{tokens1} => {n_grams1}')

In [ ]:
n_gram_finder = nltk.collocations.TrigramCollocationFinder.from_words(tokens1)

print(f'Most Common N-Gram Finder using NLTK (n = 3): \n{tokens1} => {n_gram_finder.ngram_fd.most_common(2)}')

In [ ]:
word_count = {}

for word in tokens1:
    
    if word not in word_count.keys():
        word_count[word] = 1
    else:
        word_count[word] += 1
        
print(f'Bag of Words: \n{tokens1} => {word_count}')

In [ ]:
stopwords = set(STOPWORDS)

wc = WordCloud(width = 900, height = 500, background_color = 'white', random_state = 10).generate(text[1])

plt.title('Word Cloud for Bag Of Words')
plt.imshow(wc)
plt.axis('off')
plt.show()

**Sentimental analysis plays a significant role in determining the polarity of a review or a comment. It is used to know whether the person is talking about something in a positive way or a negative way. It can be classified broadly into positive, negative, and neutral. For example, on a tourism website, a person leaves a remark stating, “There are beautiful tourist spots in Switzerland. “The word ‘beautiful’ is positive as it describes Switzerland as pretty.****

In [ ]:
sia = SentimentIntensityAnalyzer()
polarity_scores1 = sia.polarity_scores(text[10])

print(f"Sentiment Analysis using NLTK: \n{text[10]} => {polarity_scores1}")

In [ ]:
x = np.array([[sia.polarity_scores(word)['compound'] for word in word_tokenize(text[5])]])
annot = np.array([word_tokenize(text[5])])

plt.figure(figsize = (10, 3))
sns.heatmap(x, annot = annot, fmt = '')
plt.title('Heatmap for Sentiment Analysis')
plt.show()